In [1]:
!pip install -q tensorflow_model_optimization

In [2]:
from google.colab import drive
import sys


drive.mount('/content/drive')

sys.path.append('/content/drive/MyDrive/dnn_model_optimization')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from utils.data import decode_texts, load_data, OCRDataset
from copy import deepcopy
import matplotlib.pyplot as plt
from utils.tf_helpers import CER, CTCLoss, warmup_tf_model
from itertools import groupby
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import time


((train_imgs, train_abits), train_labels), ((val_imgs, val_abits), val_labels), alphabet = load_data('/content/drive/MyDrive/dnn_model_optimization/data', split=True, blank_idx=-1)

# Original model

In [4]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Методы компрессии/crnn_common_fields.h5',
                                   custom_objects={'CTCLoss': CTCLoss, 'CER': CER})

print(f'Original model #Params: {model.count_params()}')

Original model #Params: 1491438


In [5]:
warmup_tf_model(model, [(32, 50, 2), (32, 32, 400, 1)])

epochs = 10
batch_size = 128
nruns = 20

start = time.time()

for i in range(nruns):
  loss, cer = model.evaluate([val_abits, val_imgs], val_labels, verbose=0)
print(f'Time spent: {round((time.time()-start)/nruns/batch_size, 6)}, loss: {round(loss, 6)}, CER: {round(cer, 6)}')

Time spent: 0.058621, loss: 0.23869, CER: 0.002223


# Full model pruning

In [6]:
num_images = train_imgs.shape[0]
end_step = (num_images // batch_size + 1) * epochs

# Define model for pruning.
pruning_params = {'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                                           final_sparsity=0.80,
                                                                           begin_step=0,
                                                                           end_step=end_step)
}


model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(model, **pruning_params)
model_for_pruning.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=5e-4),
              loss=CTCLoss,
              metrics=[CER()])

First experiments have shown that small number of epochs may not be enough, so well fine tune for 10+ epochs. Though, we cannot use early stopping due to pre-planned number of epochs for pruning (sparsity decays through the learning). Maybe I should add few extra epochs after pruning

In [7]:
from tensorflow.keras.callbacks import EarlyStopping

model_for_pruning.fit([train_abits, train_imgs], train_labels, validation_data=([val_abits, val_imgs], val_labels),
                      batch_size=batch_size, epochs=epochs,
                      callbacks=[tfmot.sparsity.keras.UpdatePruningStep()])

Epoch 1/10
103/103 [==============================] - 78s 495ms/step - loss: 0.4587 - CER: 0.0080 - val_loss: 68.3577 - val_CER: 0.7392
Epoch 2/10
103/103 [==============================] - 36s 352ms/step - loss: 0.8236 - CER: 0.0188 - val_loss: 55.9288 - val_CER: 0.7685
Epoch 3/10
103/103 [==============================] - 33s 320ms/step - loss: 0.8558 - CER: 0.0169 - val_loss: 22.2777 - val_CER: 0.4841
Epoch 4/10
103/103 [==============================] - 35s 336ms/step - loss: 0.7273 - CER: 0.0135 - val_loss: 37.0889 - val_CER: 0.5991
Epoch 5/10
103/103 [==============================] - 35s 338ms/step - loss: 0.7168 - CER: 0.0136 - val_loss: 19.1522 - val_CER: 0.3714
Epoch 6/10
103/103 [==============================] - 35s 340ms/step - loss: 0.6666 - CER: 0.0121 - val_loss: 7.1800 - val_CER: 0.1457
Epoch 7/10
103/103 [==============================] - 33s 321ms/step - loss: 0.6404 - CER: 0.0115 - val_loss: 12.1935 - val_CER: 0.2631
Epoch 8/10
103/103 [=============================

In [8]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
model_for_export.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-4),
                         loss=CTCLoss, metrics=[CER()])

warmup_tf_model(model_for_export, [(32, 50, 2), (32, 32, 400, 1)])

start = time.time()

for i in range(nruns):
  loss, cer = model_for_export.evaluate([val_abits, val_imgs], val_labels, verbose=0, batch_size=batch_size)
print(f'Full prunned model. Time spent: {round((time.time()-start)/nruns/batch_size, 6)}, loss: {round(loss, 6)}, CER: {round(cer, 6)}')
print(f'Full prunned model #Params: {model_for_export.count_params()}')

model_for_export.save('/content/drive/MyDrive/dnn_models_optimization/weights/crnn_common_fields_full_prunned.h5')

Full prunned model. Time spent: 0.041095, loss: 0.478418, CER: 0.009486
Full prunned model #Params: 1491438


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Partial pruning

In [9]:
def layer_selection_func(layer):
  if isinstance(layer, (tf.keras.layers.Dense, tf.keras.layers.Conv2D)):
    return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
  return layer

model_for_pruning = tf.keras.models.clone_model(model, clone_function=layer_selection_func)
model_for_pruning.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=5e-4),
              loss=CTCLoss,
              metrics=[CER()])

In [10]:
model_for_pruning.fit([train_abits, train_imgs], train_labels, validation_data=([val_abits, val_imgs], val_labels),
                      batch_size=batch_size, epochs=epochs,
                      callbacks=[tfmot.sparsity.keras.UpdatePruningStep()])

Epoch 1/10
103/103 [==============================] - 53s 403ms/step - loss: 0.4194 - CER: 0.0082 - val_loss: 0.4441 - val_CER: 0.0091
Epoch 2/10
103/103 [==============================] - 34s 326ms/step - loss: 0.3100 - CER: 0.0064 - val_loss: 0.3696 - val_CER: 0.0080
Epoch 3/10
103/103 [==============================] - 34s 332ms/step - loss: 0.2693 - CER: 0.0057 - val_loss: 0.3967 - val_CER: 0.0086
Epoch 4/10
103/103 [==============================] - 34s 334ms/step - loss: 0.2411 - CER: 0.0055 - val_loss: 0.3602 - val_CER: 0.0075
Epoch 5/10
103/103 [==============================] - 35s 343ms/step - loss: 0.2126 - CER: 0.0051 - val_loss: 0.3364 - val_CER: 0.0074
Epoch 6/10
103/103 [==============================] - 32s 314ms/step - loss: 0.2015 - CER: 0.0048 - val_loss: 0.3772 - val_CER: 0.0080
Epoch 7/10
103/103 [==============================] - 34s 331ms/step - loss: 0.2015 - CER: 0.0049 - val_loss: 0.3863 - val_CER: 0.0084
Epoch 8/10
103/103 [==============================] - 3

In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
model_for_export.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-4),
                         loss=CTCLoss, metrics=[CER()])

warmup_tf_model(model_for_export, [(32, 50, 2), (32, 32, 400, 1)])

start = time.time()

for i in range(nruns):
  loss, cer = model_for_export.evaluate([val_abits, val_imgs], val_labels, verbose=0, batch_size=batch_size)
print(f'Partial prunned model. Time spent: {round((time.time()-start)/nruns/batch_size, 6)}, loss: {round(loss, 6)}, CER: {round(cer, 6)}')
print(f'Partial prunned model #Params: {model_for_export.count_params()}')

model_for_export.save('/content/drive/MyDrive/dnn_models_optimization/weights/crnn_common_fields_partial_prunned.h5')

Partial prunned model. Time spent: 0.042377, loss: 0.348688, CER: 0.007657
Partial prunned model #Params: 1491438


In [ ]:
from google.colab import runtime

runtime.unassign()